In [46]:
import pandas as pd
import requests as r
from bs4 import BeautifulSoup

In [47]:
url_list = [{"https://freakins.com/collections/mens-jean-collection?filter.v.availability=1": 10},
       {"https://freakins.com/collections/shirts-buy-2-999?filter.v.availability=1": 7},
       {"https://freakins.com/collections/mens-t-shirts?filter.v.availability=1": 6},
       {"https://freakins.com/collections/shorts-men?filter.v.availability=1": 1},
       {"https://freakins.com/collections/straight-cargo-mens?filter.v.availability=1": 5},
       {"https://freakins.com/collections/mens-jackets-1?filter.v.availability=1": 1},
       {"https://freakins.com/collections/mens-bottomwear?filter.v.availability=1": 8},
       {"https://freakins.com/collections/pants-and-trousers-women?filter.v.availability=1": 7},
       {"https://freakins.com/collections/womens-tops-tees-shirts?filter.v.availability=1":18},
       {"https://freakins.com/collections/shop-womens-jeans?filter.v.availability=1":15},
       {"https://freakins.com/collections/freakins-curve?filter.v.availability=1":8},
       {"https://freakins.com/collections/cargo-pants?filter.v.availability=1": 5},
       {"https://freakins.com/collections/womens-jackets?filter.v.availability=1": 3},
       {"https://freakins.com/collections/shorts-women?filter.v.availability=1": 4},
       {"https://freakins.com/collections/womens-jumpsuits-dungarees?filter.v.availability=1": 1},
       {"https://freakins.com/collections/skinny?filter.v.availability=1": 1}
    ]

In [48]:
def extract_prices_from_list(price_info) :
    if "Regular price" in price_info: 
        start = price_info.find("Sale price₹") + len("Sale price₹")
        end = price_info.find("Regular price")
        sale_price = price_info[start:end].strip()

        start = price_info.find("Regular price₹") + len("Regular price₹")
        regular_price = price_info[start:].strip()
    else :
        sale_price = ''
        start = price_info.find("Sale price₹") + len("Sale price₹")
        regular_price = price_info[start:].strip()
    return [sale_price, regular_price]

In [50]:
product_names = []
sale_prices = []
original_prices = []
product_primary_images = []
product_seconday_images = []
sizes = []
categories = []
product_descriptions = []
product_urls = []

parent_url = 'https://freakins.com'

for url_dict in url_list:
    for url, pages in url_dict.items():
        print(url, pages)
        for i in range(pages):
            prefix = '&page=' + str(i+1)
            if (i == 0) : prefix = ""
            url_prefix = url + prefix
            req = r.get(url_prefix)
            soup = BeautifulSoup(req.text, 'html.parser')
            product_containers = soup.find_all('product-item', class_='product-item')
            # print(product_containers[0])
            j = 0
            for product in product_containers:
                j = j+1
                categories.append(soup.find('h1', class_='heading h3').text.strip())
                product_primary_images.append(product.find('img', class_='product-item__primary-image').get('src').lstrip('//'))
                product_names.append(product.find('a', class_='product-item-meta__title').text.strip())
                print(product.find('a', class_='product-item-meta__title').text.strip())
                price_list = extract_prices_from_list(product.find('div', class_='price-list price-list--centered').text)

                sale_prices.append(price_list[0])
                original_prices.append(price_list[1])
                # print(sale_prices)
                # print(original_prices)

                child_url = product.find('a', class_='product-item__aspect-ratio aspect-ratio').get('href')
                child_url = parent_url + child_url
                req_child = r.get(child_url)
                soup_child = BeautifulSoup(req_child.text, 'html.parser')

                product_urls.append(child_url)

                child_images = []
                child_images_div = soup_child.find_all('div', class_='product__thumbnail')
                for child_div in child_images_div :
                    child_images.append(child_div.find('img').get('src').lstrip('//'))
                # print(child_images)
                product_seconday_images.append(child_images)
                product_descriptions.append(soup_child.find('div', class_='product-form__description rte').text.strip().replace('\n', ' '))
                div_siz = soup_child.find('div',class_='product-form__option-selector')
                size_available = div_siz.find_all('div', class_='block-swatch')
                size_array = []
                # if j ==1 : print(soup_child)
                for size in size_available :
                    size_array.append(size.text.strip())
                sizes.append(size_array)

df = pd.DataFrame({
    'Category': categories,
    'Product Name': product_names,
    'Product Link': product_urls,
    'Sale Price': sale_prices,
    'Original Price': original_prices,
    'Sizes': sizes,
    'Primary Image': product_primary_images,
    'Product Secondary Images': product_seconday_images,
    'Product URLs': product_urls,
    'Product Description': product_descriptions
})

df.to_csv('freakins.csv', index=False)
print("Data scraped and saved successfully!")

https://freakins.com/collections/mens-jean-collection?filter.v.availability=1 10


KeyboardInterrupt: 